# Venus GPS

Comercially availible GPS receivers are problematic for rockets. For technical reasons filters and satellite tracking loops in off the shelf GPS chips are focused on surface uses. Even for aviation grade GPS (much, more expensive!) they do not expect you to accelerate straight up at 10 g's.

Aside from the technical challenge of keeping lock under high g-load, there are legal restrictions in place for GPS as well. Usually refered to "COCOM limits", all purchased GPS units must stop giving valid data above certain speeds and altitudes.

This makes things very difficult on a rocket. Often you run into COCOM limits and acceleration errors at the same time (high accleration and speed).

## Reported Fix Information

The Venus8 chip will report how good a fix it has as it goes. We can plot both the number or locked satellites and the "Fix Mode" which will tell us "no lock", "2D fix", or various grades of full position fix modes.

In [ ]:
from numpy import subtract, divide, multiply, median, std, var, average, arange, absolute
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
import launch

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Venus 8 GPS Reciever Reported Fix ")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"#")

ax1.plot(launch.venus.time, launch.venus.num_sv, alpha=0.75, label="Number of Sats locked")
ax1.plot(launch.venus.time, launch.venus.fix_mode, lw=3, alpha=0.75, label="Reported Fix Mode (2=3D Fix, 3=3D Fix + DGPS)")

ax2 = ax1.twinx()
ax2.set_ylabel(r"Acceleration [m/s${}^2$]")
ax2.plot(launch.adis.time[200:-200], launch.adis.acc_x_filter[200:-200], 'k-', alpha=0.18, label="IMU Acceleration")
ax2.grid(b=False)
ax2.set_ylim([-50,150])

ax1.set_xlim([-5,42])
ax2.set_xlim([-5,42])
ax1.set_ylim([0,12])
ax1.legend(loc=2)
ax2.legend(loc=1)
plt.show()

At least initially, it would look like the Venus tracked the entire flight! Though, well see this is in fact not true.

In GPS we can also get reports of the "Dilution of Precision" (DOP) over time. This is related to how good a constellation you have at any moment and some errors in the psuodoranges to the satellites. Lower numbers are better.

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Venus 8 GPS Reciever DOP")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Dilution of Precision")

ax1.plot(launch.venus.time, launch.venus.GDOP, alpha=0.75, label="GDOP")
ax1.plot(launch.venus.time, launch.venus.VDOP, alpha=0.75, label="VDOP")
ax1.plot(launch.venus.time, launch.venus.HDOP, alpha=0.75, label="HDOP")

ax2 = ax1.twinx()
ax2.set_ylabel(r"Acceleration [m/s${}^2$]")
ax2.plot(launch.adis.time[200:-200], launch.adis.acc_x_filter[200:-200], 'k-', alpha=0.18, label="IMU Acceleration")
ax2.grid(b=False)
ax2.set_ylim([-50,150])

ax1.set_xlim([-5,42])
ax2.set_xlim([-5,42])
ax1.legend(loc=2)
ax2.legend(loc=1)
plt.show()

Again the Venus misleadingly reports very good results.

## Altitude

If the fix is good, we should be able to see the rocket flight in high precision. Let's look at the reported altitdue from the Venus8 GPS chip.

In [ ]:
alt_time, imualt = launch.cached_altitude()

fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Venus 8 GPS Reciever Altitude")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Altitude AGL [km]")

ax1.plot(alt_time, divide(imualt, 1000.0), 'k-', alpha=0.2, label="IMU Integrated Vertical Velocity")
ax1.plot(launch.venus.time, divide(subtract(launch.venus.altmsl, 1390), 1000.0), alpha=0.75, label="Venus8 GPS Altitude")

plt.xlim([-5,42])
plt.ylim([0,6])
ax1.legend(loc=2)
plt.show()

Oops. The venus does not report altitude for nearly the **entire** flight. There is also a mysterious instantaneous vertical bump t+31 seconds.

In [ ]:
launch_lat = average(launch.venus.latitude[5000:10000])
launch_lon = average(launch.venus.longitude[5000:10000])
print """Though we can see the position and sea level altitude of the GPS antenna just before
launch. The launch tower with the rocket on it reports at %0.8f° N, %0.8f° E and and altitude
of %0.3f meters MSL.
""" % (launch_lat, launch_lon, average(launch.venus.altmsl[5000:10000]))

## Position

We also get lat/lon pairs for the flight.

In [ ]:
from numpy import abs as npabs
from mpl_toolkits.basemap import Basemap
from basemapshoot import shoot
import json

with open('map.json', 'r') as geojson:
    geo = json.loads(geojson.read())

    roads = []
    for coll in geo['features']:
        if coll[u'properties'][u'description'] == "road":
            lat = [c[1] for c in coll[u'geometry'][u'coordinates']]
            lon = [c[0] for c in coll[u'geometry'][u'coordinates']]
            roads.append((lon, lat))

ax = plt.figure(figsize=(14,14))
plt.title("Launch Site Map With Rocket Decent On Parachute")

m = Basemap(projection='aea',
            lat_0=43.798,lon_0=-120.6517534,
            width=1800,height=1200,resolution ='c')

m.drawparallels(arange(  43.4,  44.0,0.002),labels=[1,1,0,0], color='#cccccc', dashes=(3,7))
m.drawmeridians(arange(-120.8,-120.4,0.002),labels=[0,0,0,1], color='#cccccc', dashes=(3,7))

                        
mx, my = m(-120.64904987812042, 43.798809500361756)
plt.text(mx, my, "Flight Line", color="#444444", rotation=-13, ha='center', va='center')

for road in roads:
    rx, ry = m(road[0], road[1])
    m.plot(rx,ry, color="#cccccc", alpha=0.8, lw=2.5)

def circle(lon, lat, radius):
    cx = []
    cy = []
    for azimuth in range(0, 360):
        glon2, glat2, baz = shoot(lon, lat, azimuth, radius)
        cx.append(glon2)
        cy.append(glat2)
    cx.append(cx[0])
    cy.append(cy[0])
    cx,cy = m(cx,cy)
    plt.plot(cx,cy, color="#aadddd", dashes=(15,10), alpha=0.7)
    plt.text(cx[50]+25, cy[50], "%0.1f km"%(radius), color="#aadddd", ha='left', va='top')

circle(launch_lon, launch_lat, 1)
circle(launch_lon, launch_lat, 0.5)

x, y = m(launch.venus.longitude[10000:], launch.venus.latitude[10000:])
m.plot(x,y,'b-.', alpha=0.4)
m.plot(x,y,'r.', ms=0.4, alpha=0.5)


mx, my = m(launch_lon, launch_lat)
m.plot(mx, my, '+', color="#999999", markersize=20, markeredgewidth=1.0)
plt.text(mx-5, my-5, "Launch Tower", color="#444444", rotation=45, ha='right', va='top')

altitude = subtract(launch.venus.altmsl[12600:27000], 1390.816)
for i in range(500, 5000, 500):
    idx = (npabs(altitude-i)).argmin()
    mx, my = m(launch.venus.longitude[idx+12600], launch.venus.latitude[idx+12600])
    m.plot(mx, my, '.', color="#ffcc22", markersize=10, markeredgewidth=0.001)
    plt.text(mx-15, my+0, "%d m"%i, color="#444444", rotation=0, ha='right', va='center')

plt.show()

The flight trace is in red, and gaps (thin blue line connects where the trace should cover) are where the Venus again reports the same position throughout the flight until it suddently snaps to the correct position at apogee.


## Velocity

GPS can actually determine velocity separately from postition by comparing the doppler shifts from each satellite. Here is the total velocity over time as reported by Venus

In [ ]:
vel_time, imuvel = launch.cached_velocity()

fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Venus 8 GPS Reciever Altitude")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Speed [m/s]")

ax1.plot(vel_time, absolute(imuvel), 'k-', alpha=0.2, label="IMU Integrated Vertical Velocity")
ax1.plot(launch.venus.time, launch.venus.velocity, alpha=0.75, label="Venus8 GPS Velocity Magnitude")

plt.xlim([-5,42])
#plt.ylim([0,12])
ax1.legend(loc=1)
plt.show()

Another case of bogus data. Clearly the Venus8 lost track of everything during the launch and only reconnected after the flight was over and it was starting to open parachutes.

## Liftoff Transient

It's hard to tell from the big picture charts if it started to report correctly then stopped or if it never got a chance, so we zoom into the moment of liftoff

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Venus 8 GPS Reciever Altitude")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Acceleration [m/s${}^2$], Velocity [m/s], Altitude AGL [m]")

ax1.plot(launch.adis.time, subtract(launch.adis.acc_x, 9.8), 'k-', alpha=0.2, label="Vehicle Acceleration")
ax1.plot(launch.venus.time, launch.venus.velocity, 'r.-', alpha=0.75, label="Venus8 GPS Velocity Magnitude")
ax1.plot(launch.venus.time, subtract(launch.venus.altmsl, 1388), 'b.-', alpha=0.75, label="Venus8 GPS Altitude AGL")

plt.xlim([-1,1.5])
plt.ylim([-1,5])
ax1.legend(loc=1)
plt.show()

The GPS keeps reporting the exact same velocity and altitude values for about 10 samples after the liftoff transient, then starts reporting very strange numbers. Altitude goes down, velocity goes up but only for a second before leveling off.

It's very hard to tell from all this data if the GPS actually had lock or not. There is a lot that goes on in the chip before it reports numbers so we just can't tell.


## Liftoff Time

We can also check what the exact time was at liftoff by looking at the GPS timestamps nearest the first IMU report of acceleration and interpolating. Note that Mission Elapsed Time is defined by the ubilical cable disconnect, here we find the exact time that the rocket starts moving.

In [ ]:
from scipy.interpolate import interp1d
import gpstk

# First accel timestamp
liftoff = 0
for i, a in enumerate(subtract(launch.adis.acc_x, 9.8)):
    if a > 1:
        liftoff  = launch.adis.time[i]
        #print "Exact liftoff MET:  %0.10f s" % liftoff
        break

idx = (abs(launch.venus.time - liftoff)).argmin()
#print timestamp[idx], timestamp[idx+1]
#print TOW[idx], TOW[idx+1]

f = interp1d([launch.venus.time[idx], launch.venus.time[idx+1]], [launch.venus.TOW[idx], launch.venus.TOW[idx+1]])
tow_liftoff = f(liftoff)
#print tow_liftoff

gpst_liftoff = gpstk.GPSWeekSecond(1854, float(tow_liftoff))
civil_liftoff = gpstk.CivilTime(gpst_liftoff.toCommonTime())

gps_week, gps_tow, fluff = str(gpst_liftoff).split(' ')
print """According to the Venus GPS, and using the exact CPU clock time of liftoff from the
accelerometer, the GPS time of liftoff was on GPS Week %d at %0.2f Seconds. This would be
July %02d, 2015 %2d:%02d:%0.6f UTC, (11:17 am local time).
""" % (int(gps_week), float(gps_tow), civil_liftoff.day, civil_liftoff.hour, civil_liftoff.minute, civil_liftoff.second)